In [11]:
# --- setup / parameters ---
import geopandas as gpd
import rasterio
from rasterio.sample import sample_gen

# Load your GeoJSON
gdf = gpd.read_file("Benin_settlement_properties.geojson")
gdf = gdf.to_crs("EPSG:4326")

# List of rasters to extract
rasters = {
    'GHI': 'GHI.tif',
    'Elevation': 'elevation.tif'
}

# Function to sample raster at a point
def sample_raster(raster_path, lon, lat):
    with rasterio.open(raster_path) as src:
        # transform coords to raster CRS if needed
        if src.crs.to_string() != 'EPSG:4326':
            from pyproj import Transformer
            transformer = Transformer.from_crs('EPSG:4326', src.crs.to_string(), always_xy=True)
            lon, lat = transformer.transform(lon, lat)
        for val in src.sample([(lon, lat)]):
            return val[0]

# Extract raster values
for col, raster_path in rasters.items():
    gdf[col] = gdf.apply(lambda row: sample_raster(raster_path, row['lon'], row['lat']), axis=1)


In [14]:

# --- 1. PARAMÈTRES CLÉS (Hypothèses du Modèle) ---
household_size = 5               
horizon_years = 10                
annual_demand_growth = 0.02       # Croissance annuelle de la demande (justifiée par la transition MTF)
productivity_fraction = 0.20      # Demande productive/institutionnelle
target_electrification = 0.9      # Taux d'électrification visé à la fin de l'horizon

# Tiers de consommation -> kWh/an/foyer 
#Multi-Tier Framework (MTF) développé par la Banque Mondiale (ESMAP).
tier_to_kwh = {
    1: 40,      # Niveau 1 (Éclairage de base)
    2: 200,     # Niveau 2
    3: 600,     # Niveau 3
    4: 1600,    # Niveau 4
    5: 4000     # Niveau 5 (Haute consommation)
}

# --- 2. FONCTION POUR DÉTERMINER LE NIVEAU DE CONSOMMATION (TIER) ---
# CETTE ÉTAPE EST CONSERVÉE POUR REMPLIR L'EXIGENCE DE JUSTIFIER LE NIVEAU DE SERVICE MTF
# RWI : Wealth Index
def infer_tier_simple(row):
    rwi = row.get("mean_rwi", 0.3)
    has_nl = row.get("has_nightlight", False) 
    if isinstance(has_nl, str):
        has_nl = has_nl.lower() == 'true'
    bd_pct = row.get("building_density_percent", 0.0)

    if has_nl and rwi > 0.45 and bd_pct > 10:
        return 4
    if has_nl and rwi > 0.30:
        return 3
    if rwi > 0.45 and bd_pct > 5:
        return 3
    if rwi > 0.2:
        return 2
    return 1

# Application de la fonction pour créer la colonne "tier_inferred" (Conformité MTF)
gdf["tier_inferred"] = gdf.apply(infer_tier_simple, axis=1)

# --- 3. CALCULS DE BASE POUR L'ANNÉE 0 (EN UTILISANT LA COLONNE 'demand' kWh/jour) ---
# Hypothèse : La colonne 'demand' est la demande totale (Résidentiel + Productif) actuelle.

# 1. Nouvelle colonne : Demande Totale Initiale (kWh/an)
gdf["total_demand_yr0"] = gdf["demand"] * 365.25 

# 2. Séparation de la Demande Résidentielle (pour appliquer la croissance)
# On suppose que la demande existante inclut déjà la fraction productive
gdf["residential_demand_yr0"] = gdf["total_demand_yr0"] / (1 + productivity_fraction)
gdf["productive_demand_yr0"] = gdf["total_demand_yr0"] - gdf["residential_demand_yr0"]


# --- 4. BOUCLE DE PROJECTION (CALCULS ANNUELS) ---

years = list(range(0, horizon_years + 1)) 

for y in years:
    # 1. Facteur de Croissance (justifié par la transition MTF vers un service supérieur)
    growth_factor = (1 + annual_demand_growth) ** y

    # 2. Demande Résidentielle projetée
    # On applique la croissance uniquement à la demande résidentielle de base.
    gdf[f"residential_demand_yr{y}"] = gdf["residential_demand_yr0"] * growth_factor

    # 3. Demande Productive/Institutionnelle projetée
    # Maintenue comme pourcentage de la demande résidentielle projetée.
    gdf[f"productive_demand_yr{y}"] = gdf[f"residential_demand_yr{y}"] * productivity_fraction
    
    # 4. Demande Totale projetée
    gdf[f"total_demand_yr{y}"] = gdf[f"residential_demand_yr{y}"] + gdf[f"productive_demand_yr{y}"]


# --- 5. EXPORTATION DES RÉSULTATS ---
output_filename = "resultats_demande_projetee_revisee.csv"
gdf.to_csv(output_filename, sep=';', index=False, float_format='%.4f')

In [ ]:
# Create required columns or rename
gdf['Country'] = 'Benin'
gdf['id'] = gdf['identifier']
gdf['X_deg'] = gdf['lon']
gdf['Y_deg'] = gdf['lat']
gdf['Pop'] = gdf['population']
gdf['dist_to_grid'] = gdf['dist_to_existing_planned_transmission_lines_2017']  # or another field
gdf['GridCellArea'] = gdf['buffer_area'] #not hull because reprensts the buildings
gdf['SubstationDist'] = gdf['dist_to_substations']
gdf['RoadDist'] = gdf['dist_main_road_km'] 

# Select only required columns
onsset_csv = gdf[['id','Country','village_name','Y_deg','X_deg','Pop','elec_rate','dist_to_grid','SubstationDist','RoadDist','GHI','GridCellArea','Elevation']]
onsset_csv.to_csv("OnSSET_input.csv", index=False)